# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [29]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score
from sklearn import svm



In [30]:
# Read student data
student_data = pd.read_csv("student_data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [31]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data.index) 
n_features = len(student_data.axes[1])
n_passed = len(student_data[student_data.passed == 'yes'])
n_failed = len(student_data[student_data.passed == 'no'])
grad_rate = float(n_passed) / float(n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [32]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows


Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [33]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

def preprocess_y(y):
    for index, value in y.iteritems():
        if value == "yes":
            y.loc[index] = 1
        else:
            y.loc[index] = 0
    return y.astype(int)

y_all = preprocess_y(y_all)

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


## Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [34]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train



# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

#I wrapped this in a method because later I will run multiple tests in order deal with the variability in random splits
def split_data_intro_train_test(X_all,y_all):
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(
        X_all, y_all, test_size=0.25)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data_intro_train_test(X_all,y_all)

#standardize data , important for some future classifiers
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 296 samples
Test set: 99 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [35]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it
print clf

Training SVC...
Done!
Training time (secs): 0.008
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [36]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score


def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    #print target[:]
    return f1_score(target, y_pred, pos_label=1) #changed 'yes' for 1

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.012
F1 score for training set: 0.898617511521


In [37]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.835443037975


In [38]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    training_sizes = [100,200,300]
    for x in training_sizes:
        X_train_temp = X_train[:x]
        y_train_temp = y_train[:x]
        print "Training set size: {}".format(len(X_train_temp))
        train_classifier(clf, X_train_temp, y_train_temp)
        print "F1 score for training set: {}".format(predict_labels(clf, X_train_temp, y_train_temp))
        print "F1 score for test set: {}\n".format(predict_labels(clf, X_test, y_test))

    
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [39]:
#We are getting variability in results due to the randomness of data in train/ test splits.  
# In order to smooth this out we will run this test 10 times,
# each time with a new random split for train /test.  Mean will be reported.
       
def train_classifier_silent(clf, X_train, y_train):
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    return (end - start)

def predict_labels_silent(clf, features, target):
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    
    time_to_predict = (end - start)
    F1_score = f1_score(target, y_pred, pos_label=1) #changed 'yes' for 1
    return time_to_predict, F1_score
        
def train_predict(clf, X_train, y_train, X_test, y_test, value_from_runs):
    training_sizes = [100,200,300]
    for x in training_sizes:
        X_train_temp = X_train[:x]
        y_train_temp = y_train[:x]
        value_from_runs['training_times_%s'% x].append(train_classifier_silent(clf, X_train_temp, y_train_temp))
        
        time_to_predict_train, F1_train = predict_labels_silent(clf, X_train_temp, y_train_temp)
        value_from_runs['predTime_train_%s' % x].append(time_to_predict_train)
        value_from_runs['F1_train_%s' % x].append(F1_train)
        
        time_to_predict_test, F1_test =  predict_labels_silent(clf, X_test, y_test)
        value_from_runs['predTime_test_%s' % x].append(time_to_predict_test)
        value_from_runs['F1_test_%s' % x].append(F1_test)

def get_mean_from_multiple_runs (clf, X_all, y_all, runs):
    values_from_runs = {
    'training_times_100' : [],
    'training_times_200' : [],
    'training_times_300' : [],
    'predTime_train_100' : [],
    'predTime_train_200' : [],
    'predTime_train_300' : [],
    'F1_train_100' : [],
    'F1_train_200' : [],
    'F1_train_300' :[],
    'predTime_test_100' : [],
    'predTime_test_200' : [],
    'predTime_test_300' : [],
    'F1_test_100' : [],
    'F1_test_200' : [],
    'F1_test_300' : [],
    }
    for i in range(runs):
        X_train, X_test, y_train, y_test = split_data_intro_train_test(X_all,y_all)
        train_predict(clf, X_train, y_train, X_test, y_test, values_from_runs)
    values_from_runs = pd.DataFrame(values_from_runs)
    return np.mean(values_from_runs)

In [40]:
clf = svm.SVC()
clf_mean_values = get_mean_from_multiple_runs(clf, X_all, y_all, 100)
print clf_mean_values

F1_test_100           0.796269
F1_test_200           0.799968
F1_test_300           0.800255
F1_train_100          0.873815
F1_train_200          0.872851
F1_train_300          0.867183
predTime_test_100     0.001258
predTime_test_200     0.001819
predTime_test_300     0.002228
predTime_train_100    0.001217
predTime_train_200    0.003420
predTime_train_300    0.006704
training_times_100    0.002026
training_times_200    0.004884
training_times_300    0.009327
dtype: float64


In [41]:
# sanity check on above work. I learned about stratified after I coded the above.

def sanity_check_my_homebrew_work (clf, X_all, y_all, runs):
    sss = cross_validation.StratifiedShuffleSplit(y_all, n_iter=100, test_size = .25)
    f1_scorer = make_scorer(f1_score)
    scores = cross_validation.cross_val_score(clf, X_all,  y_all, cv=sss, scoring = f1_scorer)
    print "F1_Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)

    
clf = svm.SVC()
sanity_check_my_homebrew_work(clf, X_all, y_all, 100)

F1_Score: 0.81 (+/- 0.03)


In [42]:
# TODO: Train and predict using two other models
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(10)

clf_mean_values = get_mean_from_multiple_runs(clf, X_all, y_all, 100)
print clf_mean_values

F1_test_100           0.774150
F1_test_200           0.774770
F1_test_300           0.779820
F1_train_100          0.815950
F1_train_200          0.822937
F1_train_300          0.825737
predTime_test_100     0.002152
predTime_test_200     0.002975
predTime_test_300     0.003611
predTime_train_100    0.002270
predTime_train_200    0.004764
predTime_train_300    0.008936
training_times_100    0.000996
training_times_200    0.001412
training_times_300    0.001307
dtype: float64


In [43]:
clf = neighbors.KNeighborsClassifier(10)
sanity_check_my_homebrew_work(clf, X_all, y_all, 100)

F1_Score: 0.78 (+/- 0.05)


In [44]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

clf_mean_values = get_mean_from_multiple_runs(clf, X_all, y_all, 100)
print clf_mean_values

F1_test_100           0.707672
F1_test_200           0.703483
F1_test_300           0.715090
F1_train_100          1.000000
F1_train_200          1.000000
F1_train_300          1.000000
predTime_test_100     0.000320
predTime_test_200     0.000389
predTime_test_300     0.000268
predTime_train_100    0.000343
predTime_train_200    0.000376
predTime_train_300    0.000429
training_times_100    0.001164
training_times_200    0.002005
training_times_300    0.002560
dtype: float64


In [45]:
clf = tree.DecisionTreeClassifier()
sanity_check_my_homebrew_work(clf, X_all, y_all, 100)

F1_Score: 0.71 (+/- 0.08)


# 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [24]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV

scaler = preprocessing.StandardScaler().fit(X_all)
X_all_scaled = scaler.transform(X_all)

f1_scorer = make_scorer(f1_score)

param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
  {'C': [1, 10, 100, 1000], 'kernel':['sigmoid']}
 ]

clf = GridSearchCV(svm.SVC(), param_grid, cv=10, scoring = f1_scorer)
clf.fit(X_all_scaled, y_all)

print "Grid Search Complete"

Grid Search Complete


In [25]:
print clf.grid_scores_
print ''
print "The best score is : ", clf.best_score_
print ''
print "The best parameters are: ", clf.best_params_


[mean: 0.76860, std: 0.03728, params: {'kernel': 'linear', 'C': 1}, mean: 0.76995, std: 0.03682, params: {'kernel': 'linear', 'C': 10}, mean: 0.76995, std: 0.03682, params: {'kernel': 'linear', 'C': 100}, mean: 0.76995, std: 0.03682, params: {'kernel': 'linear', 'C': 1000}, mean: 0.80302, std: 0.00299, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.001}, mean: 0.80302, std: 0.00299, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001}, mean: 0.80535, std: 0.03336, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}, mean: 0.80302, std: 0.00299, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001}, mean: 0.76832, std: 0.04351, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.001}, mean: 0.79917, std: 0.03998, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.0001}, mean: 0.71831, std: 0.09778, params: {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}, mean: 0.77954, std: 0.03259, params: {'kernel': 'rbf', 'C': 1000, 'gamma': 0.0001}, mean: 0.80302, std: 0.00299, params: {'kernel': 'sigmoid', 'C': 1}, m

In [46]:
# Training Tests with Dummy data, in order to see how training / predict times will scale on different
# algorithms

#initialize z_all , 100000 training samples of random numbers, 48 features to match X_all
Z_all = pd.DataFrame(np.random.randn(100000, 48), columns = X_all.columns)
GiantX_all = pd.concat([X_all, Z_all])

#initialize fake_answers, should produce a vector of binary 0's or 1's. ranomdly  add to y_all
fake_answers = np.random.randint(2, size= 100000)
add_to_y = pd.DataFrame(fake_answers)
gianty_all = y_all.append(add_to_y)

#data set for these tests will now be GiantX_all and gianty_all


In [48]:
#find training times on SVC with 75k samples.
X_train, X_test, y_train, y_test = split_data_intro_train_test(GiantX_all, gianty_all)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

#SVC train and print times
clf = svm.SVC()
train_classifier(clf, X_train, y_train)


Training SVC...
Done!
Training time (secs): 824.998


/usr/local/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [49]:
#SVC predict and print times, use training data for larger size.
predict_labels(clf, X_train, y_train)

Predicting labels using SVC...
Done!
Prediction time (secs): 483.422


0.73302070709731815

In [50]:
#KNN train and print times
clf = neighbors.KNeighborsClassifier(10)
train_classifier(clf, X_train, y_train)

Training KNeighborsClassifier...
Done!
Training time (secs): 0.456


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [51]:
#KNN predict and print times
predict_labels(clf, X_train, y_train)

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 1257.372


0.57656577764459094

In [52]:
#DT train and print times
clf = tree.DecisionTreeClassifier()
train_classifier(clf, X_train, y_train)

Training DecisionTreeClassifier...
Done!
Training time (secs): 14.793


In [53]:
#DT predict and print times
predict_labels(clf, X_train, y_train)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.041


1.0